In [1]:
# from pycoral.adapters import classify, common
# from pycoral.utils.edgetpu import make_interpreter
# import platform
# import tflite_runtime.interpreter as tflite

# _EDGETPU_SHARED_LIB = {
#   'Linux': 'libedgetpu.so.1',
#   'Darwin': 'libedgetpu.1.dylib',
#   'Windows': 'edgetpu.dll'
# }[platform.system()]

# print(_EDGETPU_SHARED_LIB)

# interp = tflite.load_delegate(_EDGETPU_SHARED_LIB)

# print(help(interp))

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import pandas as pd
import os


def get_mnist784_dataset():
    data_dir = os.path.join(os.getcwd(), "datasets")
    if not os.path.exists(data_dir):
        os.mkdir(data_dir)
    mnist_dir =  os.path.join(os.getcwd(), "datasets", "mnist_784")
    if not os.path.exists(mnist_dir):
        os.mkdir(mnist_dir)

    mnist = fetch_openml('mnist_784', as_frame=False)
    data_df = pd.DataFrame(mnist.data, columns=mnist.feature_names)
    target_df = pd.DataFrame(mnist.target, columns=mnist.target_names)
    data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
    target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
    data_df.to_csv(data_path)
    target_df.to_csv(target_path)
    X, y = mnist.data, mnist.target
    return X, y

def mnist784_df_from_csv():
    data_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_data.csv")
    target_path = os.path.join(os.getcwd(), "datasets", "mnist_784", "mnist_784_target.csv")
    data_df = pd.read_csv(data_path)
    target_df = pd.read_csv(target_path)
    X_df, y_df = data_df, target_df
    X_df.drop(columns="Unnamed: 0", inplace=True)
    y_df.drop(columns="Unnamed: 0", inplace=True)
    return X_df, y_df

def train_predict_mnist784():
    X_df, y_df = mnist784_df_from_csv()
    X = X_df.values
    y = y_df.values
    some_digit = X[0]
    X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
    y_train_5 = (y_train == 5)
    y_test_5 = (y_test == 5)
    sgd_clf = SGDClassifier(random_state=42)
    sgd_clf.fit(X_train, y_train_5)
    sgd_clf.predict([some_digit])
    scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")
    return scores

# res = train_predict_mnist784()
# print(res)

In [42]:
from collections import defaultdict
import cv2
import os
import re
import time
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score


def plot_digit_arr(img_data):
    shape = img_data.shape[0]
    size = int(shape**0.5)
    img = img_data.reshape(size, size)
    plt.imshow(img, cmap="binary")
    plt.axis("off")

def crop_images(paths):
    i = 0
    img_dir = os.path.join(os.getcwd(), "datasets", "images")
    if not os.path.exists(img_dir):
        os.mkdir(img_dir)
    for path in paths:
        i += 1
        new_img_path = os.path.join(img_dir, "img{}.png".format(i))
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = img_data[0:45, 0:278]
        cv2.imwrite(new_img_path, img)

def split_image_digits(path):
    new_dirname = os.path.split(path)[-1].split(".")[0]
    new_dir = os.path.join(os.getcwd(), "datasets", "newer_images")
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
    new_path = os.path.join(new_dir, new_dirname)
    if not os.path.exists(new_path):
        os.mkdir(new_path)
    img = cv2.imread(path)
    img1 = img[0:44, 0:92]
    img2 = img[0:44, 92:184]
    img3 = img[0:44, 184:276]
    cv2.imwrite(os.path.join(new_path, "image1.png"), img1)
    cv2.imwrite(os.path.join(new_path, "image2.png"), img2)
    cv2.imwrite(os.path.join(new_path, "image3.png"), img3)
    return new_path

def create_image_data(img_dirname):
    new_image_paths = []
    images_dir = os.path.join(os.getcwd(), "datasets", img_dirname)
    paths = [[os.path.join(dn, fn) for fn in files] for dn, _ , files in os.walk(images_dir)]
    paths = sum(paths, [])
    crop_images(paths)
    for path in paths:
        new_path = split_image_digits(path)
        new_image_paths.append(new_path)
    return new_image_paths
    
def preprocess_image_data(image_path, n):
    new_images = [os.path.join(image_path, fn) for fn in os.listdir(image_path)]
    img1 = cv2.imread(new_images[0], cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(new_images[1], cv2.IMREAD_GRAYSCALE)
    img3 = cv2.imread(new_images[2], cv2.IMREAD_GRAYSCALE)
    imgs = [img1.flatten(), img2.flatten(), img3.flatten()]
    dfs = {}
    for i in range(len(imgs)):
        img = imgs[i]
        name = "image{}".format((i+1)+n)
        df = img_to_dict(img)
        dfs[name] = df
    return dfs

def img_to_dict(img):
    p = 0
    res = {}
    for i in range(img.shape[0]):
        p += 1
        col = "pixel{}".format(p)
        pix = img[i]
        res[col] = pix
    return res

def images_to_df(img_path, n):
    dfs = preprocess_image_data(img_path, n)
    main_df = pd.DataFrame(dfs.values(), index=dfs.keys())
    return main_df

def save_image_data(imgs_path):
    dfs = []
    n = 0
    for img_name in os.listdir(imgs_path):
        img_path = os.path.join(imgs_path, img_name)
        df = images_to_df(img_path, n)
        dfs.append(df)
        n += 3
    main_df = pd.DataFrame()
    for df in dfs:
        if len(main_df) == 0:
            main_df = df
        else:
            main_df = pd.concat([main_df, df])
    df_save_dir = os.path.join(os.getcwd(), "output")
    if not os.path.exists(df_save_dir):
        os.mkdir(df_save_dir)
    main_df.to_csv(os.path.join(df_save_dir, "image_pixels_{}.csv".format(int(time.time()))))
    return main_df
    
def train_predict_digits(X, y, digit=1):
    X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
    y_train_digit = (y_train == digit)
    y_test_digit = (y_test == digit)
    sgd_clf = SGDClassifier(random_state=42)
    sgd_clf.fit(X_train, y_train_digit)
    sgd_clf.predict(X_test[0])
    scores = cross_val_score(sgd_clf, X_train, y_train_digit, cv=3, scoring="accuracy")
    return scores

In [43]:
new_path_list = create_image_data("images")
new_path = os.path.join(os.getcwd(), "datasets", "newer_images")

In [44]:
df = save_image_data(new_path)

In [45]:
import numpy as np

orig_targets = ["89", "121", "28", "81", "119", "28", "86", "121", "33", "85", "121", "32", "84", "33", "82", "120", "30", "82", "120", "30", "82", "121", "49", "81", "118", "60", "69", "123", "57", "72", "121", "30", "89", "121", "28", ]
targets = [int(i) for i in orig_targets]
name = "image{}"
image_idxs = []

for i in range(1, 30):
    image_idxs.append(name.format(i))


X_train_main = np.array(df.loc[image_idxs].values)
y_train_main = np.array([[x] for x in targets])
print(X_train_main[:20].shape)
print(y_train_main[:20].shape)
print(len(orig_targets))

(20, 4048)
(20, 1)
35


In [ ]:
def train_predict_digits(X, y, digit=121):
    X_train, X_test, y_train, y_test = X[:20], X[20:], y[:20], y[20:]
    y_train_digit = (y_train == digit)
    y_test_digit = (y_test == digit)
    sgd_clf = SGDClassifier(random_state=42)
    sgd_clf.fit(X_train, y_train_digit)
    sgd_clf.predict([X_test[0]])
    scores = cross_val_score(sgd_clf, X_train, y_train_digit, cv=3, scoring="accuracy")
    return scores


res = train_predict_digits(X_train_main, y_train_main)
print(res)

[0.85714286 0.85714286 0.83333333]


c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\py_repos\image_classifier\venv\lib\site-packages\sklearn\utils\validation.py:1183: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

In [40]:
df.loc["image1"]

(44,)

In [81]:
import tensorflow as tf
import numpy as np


EPOCHS = 20

X = X_train_main
y = y_train_main

X_train, X_test, y_train, y_test = X[:20], X[20:26], y[:20], y[20:26]
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)

X_train = X_train.reshape((1, 20, 4048))
X_train = np.repeat(X_train, 20, axis=0)

def build_scn(num_classes=1000):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 4048, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss=tf.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model

# def build_scn(num_classes=1000):
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 4048, 1)),
#         tf.keras.layers.MaxPooling2D((2, 2)),
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(128, activation='relu'),
#         tf.keras.layers.Dropout(0.2),
#         tf.keras.layers.Dense(num_classes),
#         tf.keras.layers.Dense(1)
#     ])
#     model.compile(optimizer='adam', loss=tf.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
#     return model

model = build_scn()
res = model.fit(X_train, y_train, epochs=EPOCHS)
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {scores[0]}")
print(f"Accuracy: {scores[1]}")

Epoch 1/20
1/1 [==============================] - 1s 955ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 338ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 345ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 333ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 318ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 317ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 0s 333ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/20
1/1 [==============================] - 0s 318ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/20
1/1 [==============================] - 0s 334ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 10/20
1/1 [=============================

ValueError: in user code:

    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\engine\training.py", line 1972, in test_function  *
        return step_function(self, iterator)
    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\engine\training.py", line 1956, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\engine\training.py", line 1944, in run_step  **
        outputs = model.test_step(data)
    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\engine\training.py", line 1850, in test_step
        y_pred = self(x, training=False)
    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\py_repos\image_classifier\venv\lib\site-packages\keras\src\engine\input_spec.py", line 298, in as

In [93]:
import tensorflow as tf


UNITS = 128
EPOCHS = 100

X = X_train_main
y = y_train_main
num_classes = len(y)
img_height = 45
img_width = 278

X_train, X_test, y_train, y_test = X[:20], X[20:25], y[:20], y[20:25]
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)


def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(UNITS, activation=tf.nn.relu, input_shape=(4048,)),
    tf.keras.layers.Dense(UNITS, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='sgd',
              loss="binary_crossentropy",
              metrics=['accuracy'])
    return model

def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(4048,)))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1000, activation='softmax'))
    #model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
res = model.fit(X_train, y_train, epochs=EPOCHS)
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {scores[0]}")
print(f"Accuracy: {scores[1]}")

Epoch 1/100
1/1 [==============================] - 0s 335ms/step - loss: 6.9061 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 6.8854 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 5ms/step - loss: 6.8560 - accuracy: 0.1000
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 6.8185 - accuracy: 0.1000
Epoch 5/100
1/1 [==============================] - 0s 14ms/step - loss: 6.7708 - accuracy: 0.1000
Epoch 6/100
1/1 [==============================] - 0s 14ms/step - loss: 6.7125 - accuracy: 0.1000
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 6.6417 - accuracy: 0.1000
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 6.5563 - accuracy: 0.1000
Epoch 9/100
1/1 [==============================] - 0s 3ms/step - loss: 6.4545 - accuracy: 0.1000
Epoch 10/100
1/1 [==============================] - 0s 16ms/step - loss: 6.3342 - accuracy: 0.1000
Epoch 11/100


In [32]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, LSTM


UNITS = 5
EPOCHS = 60

X = X_train_main
y = y_train_main
num_classes = len(y)
img_height = 45
img_width = 278

X_train, X_test, y_train, y_test = X[:20], X[20:25], y[:20], y[20:25]



def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(UNITS, activation=tf.nn.relu, input_shape=(4048,)),
    tf.keras.layers.Dense(UNITS, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer='sgd',
              loss="binary_crossentropy",
              metrics=['accuracy'])
    return model

def build_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(4048,)))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(1000, activation='softmax'))
    #model.add(tf.keras.layers.Dense(1))
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
res = model.fit(X_train, y_train, epochs=EPOCHS)
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss: {scores[0]}")
print(f"Accuracy: {scores[1]}")

Epoch 1/60
1/1 [==============================] - 0s 354ms/step - loss: 76.6851 - accuracy: 0.0000e+00
Epoch 2/60
1/1 [==============================] - 0s 14ms/step - loss: 80.1977 - accuracy: 0.0000e+00
Epoch 3/60
1/1 [==============================] - 0s 9ms/step - loss: 81.6339 - accuracy: 0.0000e+00
Epoch 4/60
1/1 [==============================] - 0s 9ms/step - loss: 69.3393 - accuracy: 0.0000e+00
Epoch 5/60
1/1 [==============================] - 0s 16ms/step - loss: 41.4612 - accuracy: 0.0000e+00
Epoch 6/60
1/1 [==============================] - 0s 11ms/step - loss: 30.2389 - accuracy: 0.0000e+00
Epoch 7/60
1/1 [==============================] - 0s 6ms/step - loss: 22.8942 - accuracy: 0.0000e+00
Epoch 8/60
1/1 [==============================] - 0s 8ms/step - loss: 13.4447 - accuracy: 0.1000
Epoch 9/60
1/1 [==============================] - 0s 15ms/step - loss: 11.8866 - accuracy: 0.0500
Epoch 10/60
1/1 [==============================] - 0s 12ms/step - loss: 9.6102 - accuracy: 0.

In [ ]:
# loss, accuracy = model.evaluate(X_test, y_test)

# print("Loss : ", loss)
# print("Accuracy : ", accuracy)
print(X_test)

[]
